# Other datasets

To also include additional files within the object store including footprints, emissions [+boundary conditions+] you can do this by accessing the underlying methods and reading in the file.

At the moment, the expected format for these files is created through standard methods from within the acrg repository.

Before starting we need to define where our object store is loacted.

In [ ]:
import os

#object_store = "/work/chxmr/objectStore" # Doesn't work with this for some reason?
object_store = "/work/rt17603/object_store"
os.environ["OPENGHG_PATH"] = object_store

### Observations

See [workflow.ipynb](workflow.ipynb#2.-Processing-data) tutorial notebook for how to add observation files to the object store, as well as how to access and use this data. For this example we need to make sure observations from the Tacolneston (TAC) site as part of the DECC network are within the Object store.

In [ ]:
site="tac" 
network="DECC"
height="100m"
domain="EUROPE"
species="co2"

In [ ]:
from openghg.modules import ObsSurface
import glob

obs_files = glob.glob("../data/DECC/tac*.dat")

ObsSurface.read_file(filepath=obs_files, data_type="CRDS", network="DECC",
                    overwrite=True)

In [ ]:
import matplotlib.pyplot as plt
from openghg.localclient import get_observations

start_date = "2012-08-01"
end_date = "2012-09-01"

obs = get_observations(site=site, species=species, start_date=start_date,
                        end_date=end_date, inlet=height, network=network)

mf = obs[0].data["mf"]
mf.plot(marker="x")

### Footprints

In [ ]:
from openghg.modules._footprints import FOOTPRINTS

fp_file_path = "../data/footprints/TAC-100magl_EUROPE_201208.nc"

site="tac" 
network="DECC"
height="100m"
domain="EUROPE"
model_params={} # no details to add yet
overwrite=True

FOOTPRINTS.read_file(fp_file_path, site, network, height,
                    domain, model_params=model_params,
                    overwrite=overwrite)

### Emissions

In [ ]:
from openghg.modules._emissions import Emissions

flux_file_path = "../data/emissions/co2-ff-edgar-bp-mth_EUROPE_2012.nc"

species="co2"
domain="EUROPE"
source="co2-ff-edgar-bp-mth"
high_time_resolution=False #???
overwrite=True

Emissions.read_file(flux_file_path, species, domain, source, 
                   high_time_resolution,
                   overwrite=overwrite)

### Footprints data merge



In [ ]:
from openghg.processing._process_footprint import footprints_data_merge

start_date = "2012-08-01"
end_date = "2012-09-01"

CombinedData = \
    footprints_data_merge(site, height, network, domain, 
                          start_date, end_date, species=species,
                          load_flux=True)
    
data = CombinedData.data
print(data)

In [ ]:
mf = data["mf"]
mf.plot(marker="x")
## Would want to be able to plot modelled mole fraction here as a comparison

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

## IDEALLY WOULD WANT TO LOAD CARTOPY HERE

fig, ax = plt.subplots()

## Plot footprints as a 2D colour map
data_fp = data["fp"].isel(time=0) # First time point
lat = data_fp.lat
lon = data_fp.lon
footprint = data_fp.values

vmin = 1e-5 # min is 0 and can't use 0 for a log scale
vmax = footprint.max()

im = ax.pcolormesh(lon, lat, footprint, 
              norm=colors.LogNorm(vmin=vmin, 
                                  vmax=vmax), shading="auto") # Put on a log scale
cb = plt.colorbar(im, ax=ax)
cb.set_label("Footprint")

## Add station position
station_lon = data.attrs["station_longitude"]
station_lat = data.attrs["station_latitude"]
ax.scatter([station_lon],[station_lat])